In [10]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator

In [3]:
load_dotenv()

True

In [4]:
model = ChatOpenAI(model='gpt-4o-mini') # Select a model that supports structured outputs

In [5]:
# Create a schema or rule. Telling the model what structure to follow in output
class EvaluationSchema(BaseModel):

    feedback: str = Field(description="Detailed feedback for the essay.")
    score: int = Field(description="Score out of 10", ge=0, le=10)

In [7]:
model_with_structured_output = model.with_structured_output(EvaluationSchema)

In [8]:
essay = """
Role of India in AI

Artificial Intelligence has become a strategic priority for India, shaping economic growth, governance, and national security. India’s AI strategy, led by NITI Aayog’s “AI for All” and the IndiaAI Mission, focuses on using AI to address population-scale challenges in healthcare, agriculture, education, mobility, and public service delivery. India’s strongest advantage is its vast STEM talent pool, which contributes significantly to global AI development while driving domestic research in NLP for Indian languages and large-scale data systems.

The country’s digital public infrastructure—such as Aadhaar, UPI, DigiLocker, and ONDC—provides a unique foundation for deploying AI at national scale. A rapidly expanding startup ecosystem and investments by major Indian enterprises further boost innovation.

However, challenges persist: limited compute capacity, dependence on imported chips, data quality issues, and the need for massive workforce upskilling. Despite these hurdles, India is well-positioned to emerge as a global leader in inclusive, responsible, and scalable AI solutions.
"""

In [9]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10:\n{essay}'
model_with_structured_output.invoke(prompt)

EvaluationSchema(feedback="The essay effectively covers the critical aspects of India's role in artificial intelligence, highlighting both its strengths and challenges. The structure is coherent, with a logical flow from strategy to execution, and the use of specific examples like NITI Aayog and the digital public infrastructure strengthens the argument. The language is formal and appropriate for the topic, demonstrating a good command of vocabulary and sentence structure. However, the essay could benefit from more detailed explanations of certain points, such as the implications of the challenges mentioned and examples of how AI is being used in the stated sectors like healthcare and education. Additionally, providing a more robust conclusion that reflects on the future outlook for India's AI journey would enhance the overall impact.", score=8)

Now We have a LLM model with us that gives us structured output according to the specified schema.

Now we are ready to create our workflow.

In [12]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str

    overall_feedback: str

    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [25]:
def evaluate_language(state: UPSCState):

    prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10:\n{state['essay']}"
    output = model_with_structured_output.invoke(prompt)

    return {
        'language_feedback': output.feedback,
        'individual_scores': [output.score]
    }

In [26]:
def evaluate_analysis(state: UPSCState):
    prompt = f"Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10:\n{state['essay']}"
    output = model_with_structured_output.invoke(prompt)

    return {
        'analysis_feedback': output.feedback,
        'individual_scores': [output.score]
    }

In [27]:
def evaluate_thought(state: UPSCState):
    prompt = f"Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10:\n{state['essay']}"
    output = model_with_structured_output.invoke(prompt)

    return {
        'clarity_feedback': output.feedback,
        'individual_scores': [output.score]
    }

In [28]:
def final_evaluation(state: UPSCState):

    # Summary Feedback
    prompt = f"Based on the following feedbacks, provide an overall feedback for the essay:\nLanguage Feedback: {state['language_feedback']}\nAnalysis Feedback: {state['analysis_feedback']}\nClarity Feedback: {state['clarity_feedback']}"
    overall_feedback = model.invoke(prompt).content
    
    # Average Score
    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])

    return {
        'overall_feedback': overall_feedback,
        'avg_score': avg_score
    }


In [31]:
# Registering the state graph
graph = StateGraph(UPSCState)

# Adding Nodes
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)

graph.add_node('final_evaluation', final_evaluation)

# Adding Edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

# Compiling the graph
workflow = graph.compile()

In [32]:
initial_state: UPSCState = {
    'essay': essay   
}

In [33]:
final_output = workflow.invoke(initial_state)
#final_state = workflow.invoke(initial_state)

In [34]:
final_output

{'essay': '\nRole of India in AI\n\nArtificial Intelligence has become a strategic priority for India, shaping economic growth, governance, and national security. India’s AI strategy, led by NITI Aayog’s “AI for All” and the IndiaAI Mission, focuses on using AI to address population-scale challenges in healthcare, agriculture, education, mobility, and public service delivery. India’s strongest advantage is its vast STEM talent pool, which contributes significantly to global AI development while driving domestic research in NLP for Indian languages and large-scale data systems.\n\nThe country’s digital public infrastructure—such as Aadhaar, UPI, DigiLocker, and ONDC—provides a unique foundation for deploying AI at national scale. A rapidly expanding startup ecosystem and investments by major Indian enterprises further boost innovation.\n\nHowever, challenges persist: limited compute capacity, dependence on imported chips, data quality issues, and the need for massive workforce upskillin